<a href="https://colab.research.google.com/github/SonDaYeong/CP1-YamPick/blob/main/efficientnet_v2m%2C_efficientnet_v2s%2C_inception%2C_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 한글 폰트 설치하기 (꼭! 설치가 완료되면 [런타임 다시 시작]을 누르고 다시 실행하기)
!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정하기
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (13.7 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
PATH = '/content/drive/MyDrive/CS/CP1/'
image_size = 150

original_datagen = ImageDataGenerator(rescale=1./255)

train_generator = original_datagen.flow_from_directory(
        PATH+'train',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

val_generator = original_datagen.flow_from_directory(
        PATH+'test',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')
        # class_mode='int')


Found 3192 images belonging to 4 classes.
Found 813 images belonging to 4 classes.


In [5]:
class_names = train_generator.class_indices
class_names = {value:key for key, value in class_names.items()}
class_names

{0: '갈비찜', 1: '갈비탕', 2: '갈치구이', 3: '갈치조림'}

In [6]:
## 적절한 batch size, epoch 수 생각
num_classes = train_generator.num_classes
# batch_size = 128
epoch = 5

### efficientnet_v2m, efficientnet_v2s, inception, MobileNet

- 기본 모델로 작업 & 4개 카테고리 한정
- 그 중, Inception, MobileNet 모델이 성능이 높았음

In [ ]:
efficientnet_v2m = tf.keras.applications.efficientnet_v2.EfficientNetV2M(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size,image_size,3)
)


214212608/214201816 [==============================] - 4s 0us/step


In [ ]:
efficientnet_v2m.summary()

Model: "efficientnetv2-m"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 150, 150, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 75, 75, 24)   648         ['rescaling[0][0]']              
                                                                                                  
 stem_bn (BatchNormalization)   (None, 75, 75, 24)   96          ['stem_conv[0][0]'

In [ ]:
efficientnet_v2s = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size,image_size,3)
)

82436096/82420632 [==============================] - 2s 0us/step


In [ ]:
efficientnet_v2s.summary()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 150, 150, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 75, 75, 24)   648         ['rescaling_1[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 75, 75, 24)   96          ['stem_conv[0][0]'

In [ ]:
inception = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size,image_size,3)
)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [ ]:
inception = Sequential([
    inception,
    Flatten(),
    # Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

inception.summary()

base_learning_rate = 0.0001
inception.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense_1 (Dense)             (None, 3)                 55299     
                                                                 
Total params: 21,858,083
Trainable params: 21,823,651
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
inception_hist = inception.fit_generator( 
  train_generator,
  validation_data=val_generator,
  epochs=epoch
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/5
75/75 [==============================] - 43s 396ms/step - loss: 0.4603 - accuracy: 0.8289 - val_loss: 1.3815 - val_accuracy: 0.7607
Epoch 2/5
75/75 [==============================] - 26s 341ms/step - loss: 0.0830 - accuracy: 0.9691 - val_loss: 0.3411 - val_accuracy: 0.9295
Epoch 3/5
75/75 [==============================] - 25s 338ms/step - loss: 0.0491 - accuracy: 0.9870 - val_loss: 0.3031 - val_accuracy: 0.9426
Epoch 4/5
75/75 [==============================] - 25s 336ms/step - loss: 0.0201 - accuracy: 0.9937 - val_loss: 0.2663 - val_accuracy: 0.9426
Epoch 5/5
75/75 [==============================] - 25s 336ms/step - loss: 0.0394 - accuracy: 0.9891 - val_loss: 0.4517 - val_accuracy: 0.9328


In [ ]:
efficientnet_v2s_ml = Sequential([
    inception,
    Flatten(),
    Dense(num_classes, activation='softmax')
])

efficientnet_v2s_ml.summary()

base_learning_rate = 0.0001
efficientnet_v2s_ml.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
                            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                            metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 3)                 21858083  
                                                                 
 flatten_2 (Flatten)         (None, 3)                 0         
                                                                 
 dense_2 (Dense)             (None, 3)                 12        
                                                                 
Total params: 21,858,095
Trainable params: 21,823,663
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
efficientnet_v2s_hist = efficientnet_v2s_ml.fit_generator( 
  train_generator,
  validation_data=val_generator,
  # batch_size = batch_size,
  epochs=epoch
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/5
 9/75 [==>...........................] - ETA: 15s - loss: 0.9708 - accuracy: 0.4659

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


75/75 [==============================] - 37s 354ms/step - loss: 0.8361 - accuracy: 0.6403 - val_loss: 0.8363 - val_accuracy: 0.6426
Epoch 2/5
75/75 [==============================] - 25s 336ms/step - loss: 0.8128 - accuracy: 0.6633 - val_loss: 0.8293 - val_accuracy: 0.6443
Epoch 3/5
75/75 [==============================] - 25s 337ms/step - loss: 0.8108 - accuracy: 0.6596 - val_loss: 0.8305 - val_accuracy: 0.6377
Epoch 4/5
75/75 [==============================] - 25s 337ms/step - loss: 0.8042 - accuracy: 0.6637 - val_loss: 0.8177 - val_accuracy: 0.6459
Epoch 5/5
75/75 [==============================] - 25s 337ms/step - loss: 0.7998 - accuracy: 0.6625 - val_loss: 0.8118 - val_accuracy: 0.6475


In [ ]:
efficientnet_v2m_ml = Sequential([
    inception,
    Flatten(),
    # Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

efficientnet_v2m_ml.summary()

base_learning_rate = 0.0001
efficientnet_v2m_ml.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


efficientnet_v2m_hist = efficientnet_v2m_ml.fit_generator( 
  train_generator,
  validation_data=val_generator,
  epochs=epoch
)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 3)                 21858083  
                                                                 
 flatten_3 (Flatten)         (None, 3)                 0         
                                                                 
 dense_3 (Dense)             (None, 3)                 12        
                                                                 
Total params: 21,858,095
Trainable params: 21,823,663
Non-trainable params: 34,432
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
30/75 [===========>..................] - ETA: 12s - loss: 1.0151 - accuracy: 0.4906

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


75/75 [==============================] - 37s 356ms/step - loss: 0.9513 - accuracy: 0.5788 - val_loss: 0.9675 - val_accuracy: 0.5574
Epoch 2/5
75/75 [==============================] - 26s 340ms/step - loss: 0.9018 - accuracy: 0.6428 - val_loss: 0.9354 - val_accuracy: 0.5934
Epoch 3/5
75/75 [==============================] - 25s 337ms/step - loss: 0.8943 - accuracy: 0.6445 - val_loss: 0.9032 - val_accuracy: 0.6344
Epoch 4/5
75/75 [==============================] - 25s 337ms/step - loss: 0.8824 - accuracy: 0.6570 - val_loss: 0.9243 - val_accuracy: 0.6000
Epoch 5/5
75/75 [==============================] - 25s 338ms/step - loss: 0.8785 - accuracy: 0.6545 - val_loss: 0.8965 - val_accuracy: 0.6279


### MobileNet

In [8]:
# 모바일넷 모델 
def mobilenet():
  mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False,
    input_shape=(image_size,image_size,3),
    weights='imagenet'
    )
  
  mobilenet.trainable = False
  mobilenet_ml = Sequential([
      mobilenet,
      Flatten(),
      # Dense(64, activation='relu'),
      Dropout(0.2),
      Dense(num_classes, activation='softmax')
  ])

  # Fine Tuning
  mobilenet.trainable = True
  fine_tune_at = 120 # Fine-tune from this layer onwards

  for layer in mobilenet.layers[:fine_tune_at]:
      layer.trainable = False

  mobilenet_ml.summary()

  base_learning_rate = 0.0001

  mobilenet_ml.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  
  return mobilenet_ml


In [11]:
## callback & EarlyStopping

epoch = 3

cur_dir = '/content/drive/MyDrive/CS/CP1/callback' # callback 데이터 저장 
ckpt_path = os.path.join(cur_dir, 'callback_ckpt')
os.makedirs(ckpt_path, exist_ok=True)

ckpt_filepath = os.path.join(ckpt_path, 'mobilenet_model_{epoch:04d}-{val_loss:.3f}-{val_accuracy:.3f}.h5')

# 특정 기준일 경우에만 저장
# 아래 코드는 val_loss가 기존 최고 성능 모델에 비해 낮아질 때만 저장
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_filepath, 
                                                   monitor='val_loss', 
                                                   save_best_only=True,
                                                   save_weights_only=False) # save_weights_only=False 모델 전체 저장

# # 모든 epoch 결과 저장
# ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath = ckpt_filepath,
#     # save_best_only = False, 
#     save_weights_only = True, # False로 설정할 경우 모델 전체 저장
#     save_freq = 'epoch')


# loss 가 3번 동안 나아지지 않으면 훈련 종료, val_loss, val_accuracy 사용 가능
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1) 

mobilenet_ml = mobilenet()
mobilenet_hist = mobilenet_ml.fit_generator(train_generator,
          validation_data=val_generator,
          epochs=epoch,
          callbacks=[ckpt_callback, early_stop_callback])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
100/100 [==============================] - 1183s 12s/step - loss: 0.7088 - accuracy: 0.7622 - val_loss: 0.8031 - val_accuracy: 0.7970
Epoch 2/10
100/100 [==============================] - 44s 440ms/step - loss: 0.1758 - accuracy: 0.9380 - val_loss: 0.7063 - val_accuracy: 0.8339
Epoch 3/10
100/100 [==============================] - 42s 415ms/step - loss: 0.0725 - accuracy: 0.9778 - val_loss: 0.9303 - val_accuracy: 0.8303
Epoch 4/10
100/100 [==============================] - 41s 415ms/step - loss: 0.0534 - accuracy: 0.9821 - val_loss: 0.6834 - val_accuracy: 0.8610
Epoch 5/10
100/100 [==============================] - 40s 401ms/step - loss: 0.0286 - accuracy: 0.9915 - val_loss: 0.6576 - val_accuracy: 0.8745
Epoch 6/10
100/100 [==============================] - 41s 408ms/step - loss: 0.0276 - accuracy: 0.9900 - val_loss: 0.7135 - val_accuracy: 0.8721
Epoch 7/10
100/100 [==============================] - 40s 402ms/step - loss: 0.0166 - accuracy: 0.9944 - val_loss: 1.1260 - val_ac

- 모델 전체를 저장하는 것보다 가중치만 저장 후 model에 적용하는 것이 좋을 것 같음

In [28]:
# callback 결과 불러오기

# 가중치만 저장
checkpoint_path = '/content/drive/MyDrive/CS/CP1/callback/callback_ckpt/mobilenet_0003-0.760-0.835.h5'
model = mobilenet()
model.load_weights(checkpoint_path)
model.evaluate_generator(val_generator)

model.fit_generator(train_generator,
                    validation_data=val_generator,
                    epochs=5,
                    callbacks=[ckpt_callback, early_stop_callback]) 


# 모델 전체 저장 
# checkpoint_path = '/content/drive/MyDrive/CS/CP1/callback/callback_ckpt/mobilenet_0001-0.587-0.846.h5'
# model = tf.keras.models.load_model(checkpoint_path)
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
#                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
#                 metrics=['accuracy']) # 옵티마이저의 상태는 유지되지 않으므로 다시 compile
# model.fit_generator(train_generator,
#                     validation_data=val_generator,
#                     epochs=epoch,
#                     callbacks=[ckpt_callback, early_stop_callback]) 

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_11 (Flatten)        (None, 32000)             0         
                                                                 
 dropout_11 (Dropout)        (None, 32000)             0         
                                                                 
 dense_11 (Dense)            (None, 4)                 128004    
                                                                 
Total params: 2,385,988
Trainable params: 1,752,900
Non-trainable params: 633,088
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


Epoch 1/5
100/100 [==============================] - 46s 426ms/step - loss: 0.0564 - accuracy: 0.9828 - val_loss: 0.9155 - val_accuracy: 0.8475
Epoch 2/5
100/100 [==============================] - 42s 420ms/step - loss: 0.0285 - accuracy: 0.9909 - val_loss: 0.9257 - val_accuracy: 0.8573
